#PySpark Setup

In [1]:
pip install -q pyspark

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [12]:
import pyspark.sql.functions as F

#import boto3 to read from S3 Bucket

In [16]:
#!pip install boto3
import os
import logging
import boto3
from botocore.exceptions import ClientError
from KEYS import ACCESS_KEY, SECRET_ACCESS_KEY

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_ACCESS_KEY
    )

def upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_name)

    try:
        res = s3_client.upload_file(file_name, bucket, object_name)
        if res:
            print("Successfully uploaded file")
    except ClientError as e:
        logging.error(e)
        return False
    return True

def download_file(file_name, bucket, object_name):
    try:
        res = s3_client.download_file(bucket, object_name, file_name)
        if res:
            print("Successfully downloaded file")
    except ClientError as e:
        logging.error(e)
        return False
    return True

#Downloading File from S3 Bucket

In [18]:
bucket = "capstone-project-team2"
file1 = "LOL_items_stats.csv"
file2 = "lol_champion_stats_en.csv"
download_file(file1, bucket=bucket, object_name=file1)
download_file(file2, bucket=bucket, object_name=file2)


True

#Reading File from CSV

In [114]:
df1 = spark.read.csv(file1,encoding='ISO-8859-1', header=True, sep =';')
df2 = spark.read.csv(file2,encoding='ISO-8859-1', header=True, sep =';')


#Data Cleaning


#Stats Dataset Cleaning

In [120]:
#changing the data types to float
from pyspark.sql.functions import *
cols = ["Cost","Sell", 'AS', 'Crit', 'LS', 'APen', 'AP','AH', 'Mana','MP5','HSP','OVamp', 'MPen', 'Health', 'Armor',"MR","HP5","MS"]
for col_name in cols:
    df1 = df1.withColumn(col_name, col(col_name).cast('float'))

#Making Item column primary key(nullable=false)
df1 = df1.withColumn("Item", F.coalesce(F.col("Item"), F.lit(0)))

#replacing null value to unknown for Maps Column
df1 = df1.na.fill(value="Unknown", subset=["Maps"])

# Displaying table
df1.printSchema()
df1.show(200)

root
 |-- Item: string (nullable = false)
 |-- Cost: float (nullable = true)
 |-- Sell: float (nullable = true)
 |-- AD: string (nullable = true)
 |-- AS: float (nullable = true)
 |-- Crit: float (nullable = true)
 |-- LS: float (nullable = true)
 |-- APen: float (nullable = true)
 |-- AP: float (nullable = true)
 |-- AH: float (nullable = true)
 |-- Mana: float (nullable = true)
 |-- MP5: float (nullable = true)
 |-- HSP: float (nullable = true)
 |-- OVamp: float (nullable = true)
 |-- MPen: float (nullable = true)
 |-- Health: float (nullable = true)
 |-- Armor: float (nullable = true)
 |-- MR: float (nullable = true)
 |-- HP5: float (nullable = true)
 |-- MS: float (nullable = true)
 |-- Maps: string (nullable = false)

+--------------------+------+------+----+----+----+----+----+-----+----+-----+----+----+-----+----+------+-----+----+----+----+-------+
|                Item|  Cost|  Sell|  AD|  AS|Crit|  LS|APen|   AP|  AH| Mana| MP5| HSP|OVamp|MPen|Health|Armor|  MR| HP5|  MS|   M

In [121]:
df2.show()

+-------------+--------------------+------------+----------+------------+-----------+------------------+--------+----------+-----+-----+--------+----+----+---------+-----+----+--------+-----+----+---------+----+----+--------+-----+------+----------+----+----+--------+----+----+--------+-----+--------+-----+-----------+
|champion_name|      champion_title|       class| playstyle|date_release|last_change|blue_essence_price|rp_price| ressource|   HP|  HP+|HP_lvl18| HP5|HP5+|HP5_lvl18|   MP| MP+|MP_lvl18|  MP5|MP5+|MP5_lvl18|  AD| AD+|AD_lvl18|   AS|   AS+|  AS_lvl18|  AR| AR+|AR_lvl18|  MR| MR+|MR_lvl18|   MS|MS_lvl18|range|range_lvl18|
+-------------+--------------------+------------+----------+------------+-----------+------------------+--------+----------+-----+-----+--------+----+----+---------+-----+----+--------+-----+----+---------+----+----+--------+-----+------+----------+----+----+--------+----+----+--------+-----+--------+-----+-----------+
|       Aatrox|    the Darkin Blade| 

#Champion Dataset Cleaning


In [122]:
# Converting champion_name to non-nullable
df2 = df2.withColumn("champion_name", F.coalesce(F.col("champion_name"), F.lit(0)))

#Changing date release to date datatype
df2 = df2.withColumn('date_release', F.to_timestamp('date_release', 'dd/MM/yyyy'))
df2 = df2.withColumn('date_release', F.to_date(col("date_release"),"dd-MM-yyyy"))

# Converting columns to double
cols = cols = [ 'HP', 'HP+', 'HP_lvl18', 'HP5', 'HP5+', 'HP5_lvl18', 'MP', 'MP+', 'MP_lvl18', 'MP5', 'MP5+', 'MP5_lvl18', 'AD', 'AD+', 'AD_lvl18', 'AS', 'AS_lvl18', 'AR', 'AR+', 'AR_lvl18', 'MR', 'MR+', 'MR_lvl18', 'MS', 'MS_lvl18', 'range', 'range_lvl18']
for col_name in cols:
    df2 = df2.withColumn(col_name, col(col_name).cast('float'))

# Displaying table
df2.printSchema()
df2.show()

root
 |-- champion_name: string (nullable = false)
 |-- champion_title: string (nullable = true)
 |-- class: string (nullable = true)
 |-- playstyle: string (nullable = true)
 |-- date_release: date (nullable = true)
 |-- last_change: string (nullable = true)
 |-- blue_essence_price: string (nullable = true)
 |-- rp_price: string (nullable = true)
 |-- ressource: string (nullable = true)
 |-- HP: float (nullable = true)
 |-- HP+: float (nullable = true)
 |-- HP_lvl18: float (nullable = true)
 |-- HP5: float (nullable = true)
 |-- HP5+: float (nullable = true)
 |-- HP5_lvl18: float (nullable = true)
 |-- MP: float (nullable = true)
 |-- MP+: float (nullable = true)
 |-- MP_lvl18: float (nullable = true)
 |-- MP5: float (nullable = true)
 |-- MP5+: float (nullable = true)
 |-- MP5_lvl18: float (nullable = true)
 |-- AD: float (nullable = true)
 |-- AD+: float (nullable = true)
 |-- AD_lvl18: float (nullable = true)
 |-- AS: float (nullable = true)
 |-- AS+: string (nullable = true)
 |-- 

#Data Exploration